In [31]:
import json
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [14]:
# 1. Load the data from the JSON file
with open('/Users/farhanshakeel/Desktop/data.json', 'r') as f:
    data = json.load(f)

In [24]:

# 2. Preprocess and split the data into training and validation sets
X = []
y = []

for label, coords in data.items():
    for coord in coords:
        X.append(np.array(coord).flatten())
        y.append(label)

X = np.array(X)
y = np.array(y)


In [25]:
# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode the labels
encoder = LabelEncoder()
y = encoder.fit_transform(y)


In [26]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [27]:
# 3. Create a suitable model architecture using TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(29, activation='softmax')
])


In [28]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [29]:
# 4. Train the model using the training data
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), batch_size=32)


Epoch 1/50
723/723 [==============================] - 7s 9ms/step - loss: 1.8715 - accuracy: 0.4959 - val_loss: 0.6550 - val_accuracy: 0.8406
Epoch 2/50
723/723 [==============================] - 7s 9ms/step - loss: 0.6712 - accuracy: 0.8089 - val_loss: 0.3858 - val_accuracy: 0.8970
Epoch 3/50
723/723 [==============================] - 7s 10ms/step - loss: 0.4594 - accuracy: 0.8674 - val_loss: 0.3024 - val_accuracy: 0.9153
Epoch 4/50
723/723 [==============================] - 7s 10ms/step - loss: 0.3851 - accuracy: 0.8885 - val_loss: 0.2644 - val_accuracy: 0.9293
Epoch 5/50
723/723 [==============================] - 7s 10ms/step - loss: 0.3405 - accuracy: 0.9028 - val_loss: 0.2509 - val_accuracy: 0.9262
Epoch 6/50
723/723 [==============================] - 7s 10ms/step - loss: 0.3210 - accuracy: 0.9071 - val_loss: 0.2262 - val_accuracy: 0.9433
Epoch 7/50
723/723 [==============================] - 7s 9ms/step - loss: 0.3056 - accuracy: 0.9114 - val_loss: 0.2251 - val_accuracy: 0.9379
Ep

In [30]:
# 5. Evaluate the model using the validation data
_, accuracy = model.evaluate(X_val, y_val)
print("Accuracy: {:.2f}%".format(accuracy * 100))

181/181 [==============================] - 1s 5ms/step - loss: 0.1203 - accuracy: 0.9666
Accuracy: 96.66%


In [36]:
from sklearn.metrics import classification_report

# Obtain the predicted labels for the validation data
y_val_pred = model.predict(X_val)
y_val_pred_labels = np.argmax(y_val_pred, axis=1)

# Decode the labels back to the original class names
y_val_true_labels = encoder.inverse_transform(y_val)
y_val_pred_labels_decoded = encoder.inverse_transform(y_val_pred_labels)

# Print the classification report
print(classification_report(y_val_true_labels, y_val_pred_labels_decoded))


181/181 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

           A       1.00      0.98      0.99       458
           B       0.99      1.00      0.99       291
           C       1.00      0.96      0.98        24
           D       0.99      0.97      0.98       148
           E       0.98      0.99      0.99       391
           F       1.00      0.96      0.98       255
           G       0.99      1.00      0.99       459
           H       0.99      1.00      0.99       216
           I       1.00      0.97      0.98       187
           J       1.00      0.97      0.98       275
           K       1.00      0.97      0.99       457
           L       0.99      0.98      0.98       438
           M       0.74      0.93      0.83        84
           N       0.94      0.79      0.86        91
           O       0.92      1.00      0.96        58
           P       1.00      0.79      0.88        29
           Q       0.81   

In [35]:
model.save("final_asl3.h5")

In [32]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [33]:
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)